In [2]:
%pip install folium scikit-learn numpy pandas matplotlib seaborn

  Using cached scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.15.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-macosx_11_0_arm64.whl.metadat

In [3]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


# def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
#     philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

#     for base in bases:
#         folium.Marker(
#             location=[base.latitude, base.longitude],
#             popup=f"{base.city_name} Base",
#             icon=folium.Icon(color="blue", icon="home")
#         ).add_to(philippines_map)

#     for ticket in tickets:
#         color = "green" if ticket.assigned else "red"
#         folium.Marker(
#             location=[ticket.latitude, ticket.longitude],
#             popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi}",
#             icon=folium.Icon(color=color, icon="leaf")
#         ).add_to(philippines_map)

#     philippines_map.save(filename)

def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    # Create base map
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    # Add drone bases
    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    # Add tickets
    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        # If the ticket is assigned and resolved, draw a line from base to ticket
        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.home_base.latitude, drone.home_base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    # Save to HTML
    philippines_map.save(filename)



# def export_to_csv(tickets, bases):
#     ticket_data = [{
#         "sample_id": t.sample_id,
#         "latitude": t.latitude,
#         "longitude": t.longitude,
#         "ndvi": t.ndvi,
#         "assigned": t.assigned,
#         "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
#         "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
#     } for t in tickets]

#     df_tickets = pd.DataFrame(ticket_data)
#     df_tickets.to_csv("ticket_assignments.csv", index=False)

#     drone_data = []
#     for base in bases:
#         for d in base.drones:
#             drone_data.append({
#                 "drone_id": d.drone_id,
#                 "status": d.status,
#                 "base": d.base.city_name,
#                 "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
#             })

#     df_drones = pd.DataFrame(drone_data)
#     df_drones.to_csv("drone_status.csv", index=False)

def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    import pandas as pd

    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)

def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")

In [6]:
import time

def main():
    # Step 1: Create 10 drone bases across the Philippines
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(2):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Step 2: Create 100 crop tickets randomly
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        ticket = Ticket(f"T{i+1}", lat, lon, ndvi)
        tickets.append(ticket)

    # Step 3: Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        # Assign tickets
        assign_tickets_to_drones(bases, tickets)

        # Visualize map and show base → resolved ticket lines
        visualize_assignments(bases, tickets, filename=f"drone_ticket_map_step{step+1}.html")
        print(f"Map saved: drone_ticket_map_step{step+1}.html")

        # Export CSV status
        export_to_csv(tickets, bases, filename=f"drone_status_step{step+1}.csv")
        print(f"CSV exported: drone_status_step{step+1}.csv")

        # Simulate step progress
        simulate_time_step(bases)

        # Optional delay to simulate live update
        time.sleep(1)

    print("\n✅ Simulation complete!")

if __name__ == "__main__":
    main()



--- Time Step 1 ---
Map saved: drone_ticket_map_step1.html


TypeError: export_to_csv() got an unexpected keyword argument 'filename'

## work

In [8]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


# def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
#     philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

#     for base in bases:
#         folium.Marker(
#             location=[base.latitude, base.longitude],
#             popup=f"{base.city_name} Base",
#             icon=folium.Icon(color="blue", icon="home")
#         ).add_to(philippines_map)

#     for ticket in tickets:
#         color = "green" if ticket.assigned else "red"
#         folium.Marker(
#             location=[ticket.latitude, ticket.longitude],
#             popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi}",
#             icon=folium.Icon(color=color, icon="leaf")
#         ).add_to(philippines_map)

#     philippines_map.save(filename)

def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    # Create base map
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    # Add drone bases
    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    # Add tickets
    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        # If the ticket is assigned and resolved, draw a line from base to ticket
        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.home_base.latitude, drone.home_base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    # Save to HTML
    philippines_map.save(filename)



# def export_to_csv(tickets, bases):
#     ticket_data = [{
#         "sample_id": t.sample_id,
#         "latitude": t.latitude,
#         "longitude": t.longitude,
#         "ndvi": t.ndvi,
#         "assigned": t.assigned,
#         "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
#         "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
#     } for t in tickets]

#     df_tickets = pd.DataFrame(ticket_data)
#     df_tickets.to_csv("ticket_assignments.csv", index=False)

#     drone_data = []
#     for base in bases:
#         for d in base.drones:
#             drone_data.append({
#                 "drone_id": d.drone_id,
#                 "status": d.status,
#                 "base": d.base.city_name,
#                 "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
#             })

#     df_drones = pd.DataFrame(drone_data)
#     df_drones.to_csv("drone_status.csv", index=False)

def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    import pandas as pd

    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)

def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")

In [10]:
import time

def main():
    # Step 1: Create 10 drone bases across the Philippines
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(2):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Step 2: Create 100 crop tickets randomly
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        ticket = Ticket(f"T{i+1}", lat, lon, ndvi)
        tickets.append(ticket)

    # Step 3: Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        # Assign tickets
        assign_tickets_to_drones(bases, tickets)

        # Visualize map and show base → resolved ticket lines
        visualize_assignments(bases, tickets, filename=f"drone_ticket_map_step{step+1}.html")
        print(f"Map saved: drone_ticket_map_step{step+1}.html")

        # Export CSV status
        export_to_csv(tickets, bases, ticket_filename=f"drone_status_step{step+1}.csv")
        print(f"CSV exported: drone_status_step{step+1}.csv")

        # Simulate step progress
        simulate_time_step(bases)

        # Optional delay to simulate live update
        time.sleep(1)

    print("\n✅ Simulation complete!")

if __name__ == "__main__":
    main()



--- Time Step 1 ---
Map saved: drone_ticket_map_step1.html
CSV exported: drone_status_step1.csv

⏱ Time Step 1
🛰 Drone DR1 from Manila completed mission.
🛰 Drone DR2 from Manila completed mission.
🛰 Drone DR3 from Cebu completed mission.
🛰 Drone DR4 from Cebu completed mission.
🛰 Drone DR5 from Davao completed mission.
🛰 Drone DR6 from Davao completed mission.
🛰 Drone DR7 from Baguio completed mission.
🛰 Drone DR8 from Baguio completed mission.
🛰 Drone DR9 from Zamboanga completed mission.
🛰 Drone DR10 from Zamboanga completed mission.
🛰 Drone DR11 from Iloilo completed mission.
🛰 Drone DR12 from Iloilo completed mission.
🛰 Drone DR13 from General Santos completed mission.
🛰 Drone DR14 from General Santos completed mission.
🛰 Drone DR15 from Tuguegarao completed mission.
🛰 Drone DR16 from Tuguegarao completed mission.
🛰 Drone DR17 from Legazpi completed mission.
🛰 Drone DR18 from Legazpi completed mission.
🛰 Drone DR19 from Puerto Princesa completed mission.
🛰 Drone DR20 from Puerto P

### Marouane WOrk

In [11]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import time

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone
        self.resolved_by = drone  # Added to support map polyline


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.base.latitude, drone.base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    philippines_map.save(filename)


def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)


def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")


def main():
    # Step 1: Create 10 drone bases across the Philippines
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(2):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Step 2: Create 100 crop tickets randomly
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        ticket = Ticket(f"T{i+1}", lat, lon, ndvi)
        tickets.append(ticket)

    # Step 3: Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        # Assign tickets
        assign_tickets_to_drones(bases, tickets)

        # Visualize assignments
        visualize_assignments(bases, tickets, filename=f"drone_ticket_map_step{step+1}.html")
        print(f"Map saved: drone_ticket_map_step{step+1}.html")

        # Export status CSVs
        export_to_csv(
            tickets,
            bases,
            ticket_filename=f"ticket_assignments_step{step+1}.csv",
            drone_filename=f"drone_status_step{step+1}.csv"
        )
        print(f"CSV exported: ticket_assignments_step{step+1}.csv and drone_status_step{step+1}.csv")

        # Simulate completion of missions
        simulate_time_step(bases, step+1)

        # Optional delay
        time.sleep(1)

    print("\n✅ Simulation complete!")


if __name__ == "__main__":
    main()



--- Time Step 1 ---
Map saved: drone_ticket_map_step1.html
CSV exported: ticket_assignments_step1.csv and drone_status_step1.csv

⏱ Time Step 1
🛰 Drone DR1 from Manila completed mission.
🛰 Drone DR2 from Manila completed mission.
🛰 Drone DR3 from Cebu completed mission.
🛰 Drone DR4 from Cebu completed mission.
🛰 Drone DR5 from Davao completed mission.
🛰 Drone DR6 from Davao completed mission.
🛰 Drone DR7 from Baguio completed mission.
🛰 Drone DR8 from Baguio completed mission.
🛰 Drone DR9 from Zamboanga completed mission.
🛰 Drone DR10 from Zamboanga completed mission.
🛰 Drone DR11 from Iloilo completed mission.
🛰 Drone DR12 from Iloilo completed mission.
🛰 Drone DR13 from General Santos completed mission.
🛰 Drone DR14 from General Santos completed mission.
🛰 Drone DR15 from Tuguegarao completed mission.
🛰 Drone DR16 from Tuguegarao completed mission.
🛰 Drone DR17 from Legazpi completed mission.
🛰 Drone DR18 from Legazpi completed mission.
🛰 Drone DR19 from Puerto Princesa completed mi

In [13]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


# def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
#     philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

#     for base in bases:
#         folium.Marker(
#             location=[base.latitude, base.longitude],
#             popup=f"{base.city_name} Base",
#             icon=folium.Icon(color="blue", icon="home")
#         ).add_to(philippines_map)

#     for ticket in tickets:
#         color = "green" if ticket.assigned else "red"
#         folium.Marker(
#             location=[ticket.latitude, ticket.longitude],
#             popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi}",
#             icon=folium.Icon(color=color, icon="leaf")
#         ).add_to(philippines_map)

#     philippines_map.save(filename)

def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    # Create base map
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    # Add drone bases
    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    # Add tickets
    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        # If the ticket is assigned and resolved, draw a line from base to ticket
        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.home_base.latitude, drone.home_base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    # Save to HTML
    philippines_map.save(filename)



# def export_to_csv(tickets, bases):
#     ticket_data = [{
#         "sample_id": t.sample_id,
#         "latitude": t.latitude,
#         "longitude": t.longitude,
#         "ndvi": t.ndvi,
#         "assigned": t.assigned,
#         "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
#         "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
#     } for t in tickets]

#     df_tickets = pd.DataFrame(ticket_data)
#     df_tickets.to_csv("ticket_assignments.csv", index=False)

#     drone_data = []
#     for base in bases:
#         for d in base.drones:
#             drone_data.append({
#                 "drone_id": d.drone_id,
#                 "status": d.status,
#                 "base": d.base.city_name,
#                 "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
#             })

#     df_drones = pd.DataFrame(drone_data)
#     df_drones.to_csv("drone_status.csv", index=False)

def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    import pandas as pd

    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)

def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")

In [14]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import time

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone
        self.resolved_by = drone  # Added to support map polyline


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.base.latitude, drone.base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    philippines_map.save(filename)


def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)


def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")


def main():
    # Step 1: Create 10 drone bases across the Philippines
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(2):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Step 2: Create 100 crop tickets randomly
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        ticket = Ticket(f"T{i+1}", lat, lon, ndvi)
        tickets.append(ticket)

    # Step 3: Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        # Assign tickets
        assign_tickets_to_drones(bases, tickets)

        # Visualize assignments
        visualize_assignments(bases, tickets, filename=f"drone_ticket_map_step{step+1}.html")
        print(f"Map saved: drone_ticket_map_step{step+1}.html")

        # Export status CSVs
        export_to_csv(
            tickets,
            bases,
            ticket_filename=f"ticket_assignments_step{step+1}.csv",
            drone_filename=f"drone_status_step{step+1}.csv"
        )
        print(f"CSV exported: ticket_assignments_step{step+1}.csv and drone_status_step{step+1}.csv")

        # Simulate completion of missions
        simulate_time_step(bases, step+1)

        # Optional delay
        time.sleep(1)

    print("\n✅ Simulation complete!")


if __name__ == "__main__":
    main()



--- Time Step 1 ---
Map saved: drone_ticket_map_step1.html
CSV exported: ticket_assignments_step1.csv and drone_status_step1.csv

⏱ Time Step 1
🛰 Drone DR1 from Manila completed mission.
🛰 Drone DR2 from Manila completed mission.
🛰 Drone DR3 from Cebu completed mission.
🛰 Drone DR4 from Cebu completed mission.
🛰 Drone DR5 from Davao completed mission.
🛰 Drone DR6 from Davao completed mission.
🛰 Drone DR7 from Baguio completed mission.
🛰 Drone DR8 from Baguio completed mission.
🛰 Drone DR9 from Zamboanga completed mission.
🛰 Drone DR10 from Zamboanga completed mission.
🛰 Drone DR11 from Iloilo completed mission.
🛰 Drone DR12 from Iloilo completed mission.
🛰 Drone DR13 from General Santos completed mission.
🛰 Drone DR14 from General Santos completed mission.
🛰 Drone DR15 from Tuguegarao completed mission.
🛰 Drone DR16 from Tuguegarao completed mission.
🛰 Drone DR17 from Legazpi completed mission.
🛰 Drone DR18 from Legazpi completed mission.
🛰 Drone DR19 from Puerto Princesa completed mi

In [ ]:
print('hello')

hello


## final version

In [ ]:
# drone_ticket_system.py

import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import time

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None

    def assign_ticket(self, ticket):
        self.status = "in_mission"
        self.assigned_ticket = ticket
        ticket.assign(self)

    def complete_mission(self):
        self.status = "available"
        self.assigned_ticket = None


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone
        self.resolved_by = drone  # Added to support map polyline


# ----- Core Logic -----

def assign_tickets_to_drones(bases, tickets):
    for ticket in tickets:
        if not ticket.assigned:
            nearest_base = min(
                bases,
                key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
            )
            available_drone = nearest_base.get_available_drone()
            if available_drone:
                available_drone.assign_ticket(ticket)


def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                print(f"🛰 Drone {drone.drone_id} from {base.city_name} completed mission.")
                drone.complete_mission()


def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.base.latitude, drone.base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    philippines_map.save(outfile='data_logs/'+filename)


def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv('data_logs/'+ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv('data_logs/'+drone_filename, index=False)


def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")


def main():
    # Step 1: Create 10 drone bases across the Philippines
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(10):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Step 2: Create 100 crop tickets randomly
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        ticket = Ticket(f"T{i+1}", lat, lon, ndvi)
        tickets.append(ticket)

    # Step 3: Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        # Assign tickets
        assign_tickets_to_drones(bases, tickets)

        # Visualize assignments
        visualize_assignments(bases, tickets, filename=f"drone_ticket_map_step{step+1}.html")
        print(f"Map saved: drone_ticket_map_step{step+1}.html")

        # Export status CSVs
        export_to_csv(
            tickets,
            bases,
            ticket_filename=f"ticket_assignments_step{step+1}.csv",
            drone_filename=f"drone_status_step{step+1}.csv"
        )
        print(f"CSV exported: ticket_assignments_step{step+1}.csv and drone_status_step{step+1}.csv")

        # Simulate completion of missions
        simulate_time_step(bases, step+1)

        # Optional delay
        time.sleep(1)

    print("\n✅ Simulation complete!")


if __name__ == "__main__":
    main()



--- Time Step 1 ---
Map saved: drone_ticket_map_step1.html
CSV exported: ticket_assignments_step1.csv and drone_status_step1.csv

⏱ Time Step 1
🛰 Drone DR1 from Manila completed mission.
🛰 Drone DR2 from Manila completed mission.
🛰 Drone DR3 from Manila completed mission.
🛰 Drone DR4 from Manila completed mission.
🛰 Drone DR5 from Manila completed mission.
🛰 Drone DR6 from Manila completed mission.
🛰 Drone DR7 from Manila completed mission.
🛰 Drone DR8 from Manila completed mission.
🛰 Drone DR9 from Manila completed mission.
🛰 Drone DR10 from Manila completed mission.
🛰 Drone DR11 from Cebu completed mission.
🛰 Drone DR12 from Cebu completed mission.
🛰 Drone DR13 from Cebu completed mission.
🛰 Drone DR14 from Cebu completed mission.
🛰 Drone DR15 from Cebu completed mission.
🛰 Drone DR16 from Cebu completed mission.
🛰 Drone DR17 from Cebu completed mission.
🛰 Drone DR18 from Cebu completed mission.
🛰 Drone DR19 from Cebu completed mission.
🛰 Drone DR20 from Cebu completed mission.
🛰 Dr

In [ ]:
import folium
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import time
import math

# ----- Data Classes -----

class Drone:
    def __init__(self, drone_id, base, max_missions=3):
        self.drone_id = drone_id
        self.base = base
        self.status = "available"
        self.assigned_ticket = None
        self.mission_count = 0
        self.max_missions = max_missions
        self.mission_history = []

    def assign_ticket(self, ticket):
        if self.status == "available":
            self.status = "in_mission"
            self.assigned_ticket = ticket
            ticket.assign(self)
            print(f"🛰 Drone {self.drone_id} assigned to Ticket {ticket.sample_id}")
        else:
            print(f"⚠️ Drone {self.drone_id} not available for assignment.")

    def complete_mission(self):
        if self.status == "in_mission":
            self.mission_history.append(self.assigned_ticket.sample_id)
            self.mission_count += 1
            print(f"✅ Drone {self.drone_id} completed mission for Ticket {self.assigned_ticket.sample_id}")
            self.assigned_ticket = None

            if self.mission_count >= self.max_missions:
                self.status = "maintenance"
                print(f"🛠 Drone {self.drone_id} is now under maintenance.")
            else:
                self.status = "available"

    def reset_maintenance(self):
        if self.status == "maintenance":
            self.mission_count = 0
            self.status = "available"
            print(f"🔧 Drone {self.drone_id} has been maintained and is available again.")


class DroneBase:
    def __init__(self, city_name, latitude, longitude):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones = []

    def add_drone(self, drone):
        self.drones.append(drone)

    def get_available_drone(self):
        for drone in self.drones:
            if drone.status == "available":
                return drone
        return None


class Ticket:
    def __init__(self, sample_id, latitude, longitude, ndvi):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone = None
        self.resolved_by = None    

    def assign(self, drone):
        self.assigned = True
        self.assigned_drone = drone
        self.resolved_by = drone



class GeoCalculator:
    EARTH_RADIUS_KM = 6371.0

    @staticmethod
    def calculate_distance(lat1, lon1, lat2, lon2):
        lat1_rad = GeoCalculator.degrees_to_radians(lat1)
        lon1_rad = GeoCalculator.degrees_to_radians(lon1)
        lat2_rad = GeoCalculator.degrees_to_radians(lat2)
        lon2_rad = GeoCalculator.degrees_to_radians(lon2)

        delta_lat = lat2_rad - lat1_rad
        delta_lon = lon2_rad - lon1_rad

        a = (math.sin(delta_lat / 2) ** 2 +
             math.cos(lat1_rad) * math.cos(lat2_rad) *
             math.sin(delta_lon / 2) ** 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return GeoCalculator.EARTH_RADIUS_KM * c

    @staticmethod
    def degrees_to_radians(degrees):
        return degrees * math.pi / 180.0


# ----- Core Logic -----

# def assign_tickets_to_drones(bases, tickets):
#     for ticket in tickets:
#         if not ticket.assigned:
#             nearest_base = min(
#                 bases,
#                 key=lambda base: (ticket.latitude - base.latitude) ** 2 + (ticket.longitude - base.longitude) ** 2
#             )
#             available_drone = nearest_base.get_available_drone()
#             if available_drone:
#                 available_drone.assign_ticket(ticket)
#             else:
#                 print(f"❌ No available drone at {nearest_base.city_name} for Ticket {ticket.sample_id}")

def assign_tickets_to_drones(bases, tickets):
    # Sort tickets by NDVI (lowest first = most critical)
    unassigned_tickets = sorted(
        [t for t in tickets if not t.assigned],
        key=lambda t: t.ndvi
    )

    for ticket in unassigned_tickets:
        best_drone = None
        best_base = None
        min_distance = float("inf")

        for base in bases:
            for drone in base.drones:
                if drone.status == "available":
                    # Calculate real-world distance using Haversine formula
                    distance = GeoCalculator.calculate_distance(
                        base.latitude, base.longitude,
                        ticket.latitude, ticket.longitude
                    )
                    if distance < min_distance:
                        min_distance = distance
                        best_drone = drone
                        best_base = base

        if best_drone:
            best_drone.assign_ticket(ticket)
            print(f"✅ Drone from {best_base.city_name} assigned to Ticket {ticket.sample_id} ({min_distance:.2f} km)")
        else:
            print(f"❌ No available drone for Ticket {ticket.sample_id}")



def simulate_time_step(bases, step=1):
    print(f"\n⏱ Time Step {step}")
    for base in bases:
        for drone in base.drones:
            if drone.status == "in_mission":
                drone.complete_mission()


def visualize_assignments(bases, tickets, filename="drone_ticket_map.html"):
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    for base in bases:
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=f"Base: {base.city_name}",
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)

    for ticket in tickets:
        color = "green" if ticket.assigned else "red"
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=f"Ticket {ticket.sample_id}\nNDVI: {ticket.ndvi:.2f}",
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)

        if ticket.assigned and ticket.resolved_by is not None:
            drone = ticket.resolved_by
            base_lat, base_lon = drone.base.latitude, drone.base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="orange",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} -> {ticket.sample_id}"
            ).add_to(philippines_map)

    philippines_map.save(filename)


def export_to_csv(tickets, bases, ticket_filename="ticket_assignments.csv", drone_filename="drone_status.csv"):
    ticket_data = [{
        "sample_id": t.sample_id,
        "latitude": t.latitude,
        "longitude": t.longitude,
        "ndvi": t.ndvi,
        "assigned": t.assigned,
        "assigned_drone_id": t.assigned_drone.drone_id if t.assigned else None,
        "assigned_base": t.assigned_drone.base.city_name if t.assigned else None
    } for t in tickets]

    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(ticket_filename, index=False)

    drone_data = []
    for base in bases:
        for d in base.drones:
            drone_data.append({
                "drone_id": d.drone_id,
                "status": d.status,
                "base": d.base.city_name,
                "assigned_ticket": d.assigned_ticket.sample_id if d.assigned_ticket else None,
                "missions_done": d.mission_count,
                "mission_history": ", ".join(d.mission_history)
            })

    df_drones = pd.DataFrame(drone_data)
    df_drones.to_csv(drone_filename, index=False)


def optimize_base_locations(tickets, n_clusters=4):
    ticket_coords = np.array([[t.latitude, t.longitude] for t in tickets])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(ticket_coords)

    print("\n📌 Suggested Base Coordinates:")
    for idx, (lat, lon) in enumerate(kmeans.cluster_centers_):
        print(f"Base {idx+1}: Lat {lat:.4f}, Lon {lon:.4f}")


def main():
    base_locations = [
        ("Manila", 14.5995, 120.9842),
        ("Cebu", 10.3157, 123.8854),
        ("Davao", 7.1907, 125.4553),
        ("Baguio", 16.4023, 120.5960),
        ("Zamboanga", 6.9214, 122.0790),
        ("Iloilo", 10.7202, 122.5621),
        ("General Santos", 6.1164, 125.1716),
        ("Tuguegarao", 17.6131, 121.7269),
        ("Legazpi", 13.1391, 123.7438),
        ("Puerto Princesa", 9.7392, 118.7353)
    ]

    bases = []
    drone_id = 1
    for name, lat, lon in base_locations:
        base = DroneBase(name, lat, lon)
        for _ in range(10):  # 2 drones per base
            drone = Drone(f"DR{drone_id}", base)
            base.add_drone(drone)
            drone_id += 1
        bases.append(base)

    # Create 100 random tickets
    np.random.seed(42)
    tickets = []
    for i in range(100):
        lat = np.random.uniform(6.0, 18.0)
        lon = np.random.uniform(120.0, 126.0)
        ndvi = np.random.uniform(0.2, 0.9)
        tickets.append(Ticket(f"T{i+1}", lat, lon, ndvi))

    # Simulate 10 time steps
    for step in range(10):
        print(f"\n--- Time Step {step+1} ---")

        assign_tickets_to_drones(bases, tickets)
        visualize_assignments(bases, tickets, filename=f"drone_test1_ticket_map_step{step+1}.html")
        export_to_csv(tickets, bases,
                      ticket_filename=f"ticket_assignments_step{step+1}.csv",
                      drone_filename=f"drone_status_step{step+1}.csv")
        simulate_time_step(bases, step+1)

        # Optional maintenance reset every 5 steps
        if (step + 1) % 5 == 0:
            for base in bases:
                for drone in base.drones:
                    if drone.status == "maintenance":
                        drone.reset_maintenance()

        time.sleep(1)

    print("\n✅ Simulation complete!")


if __name__ == "__main__":
    main()



--- Time Step 1 ---
🛰 Drone DR81 assigned to Ticket T43
✅ Drone from Legazpi assigned to Ticket T43 (334.66 km)
🛰 Drone DR82 assigned to Ticket T33
✅ Drone from Legazpi assigned to Ticket T33 (160.07 km)
🛰 Drone DR11 assigned to Ticket T97
✅ Drone from Cebu assigned to Ticket T97 (104.43 km)
🛰 Drone DR83 assigned to Ticket T10
✅ Drone from Legazpi assigned to Ticket T10 (109.61 km)
🛰 Drone DR1 assigned to Ticket T28
✅ Drone from Manila assigned to Ticket T28 (164.84 km)
🛰 Drone DR2 assigned to Ticket T11
✅ Drone from Manila assigned to Ticket T11 (145.61 km)
🛰 Drone DR84 assigned to Ticket T26
✅ Drone from Legazpi assigned to Ticket T26 (202.75 km)
🛰 Drone DR21 assigned to Ticket T23
✅ Drone from Davao assigned to Ticket T23 (90.05 km)
🛰 Drone DR12 assigned to Ticket T100
✅ Drone from Cebu assigned to Ticket T100 (192.59 km)
🛰 Drone DR85 assigned to Ticket T19
✅ Drone from Legazpi assigned to Ticket T19 (193.58 km)
🛰 Drone DR86 assigned to Ticket T55
✅ Drone from Legazpi assigned to T